# 數據整理:join, combine, reshape

## 階層式索引
    - 分類欄或列

In [ ]:
import numpy as np
import pandas as pd

data = pd.Series(np.random.randn(9),
                index=[['a', 'a', 'a', 'b', 'b', 'c', 'c', 'd', 'd'],
                       [1, 2, 3, 1, 2, 3, 1, 2, 3]])
data
'''
Out[1]: 
a  1    0.205758
   2    0.914465
   3   -1.129890
b  1    0.841650
   2   -0.571143
c  3    0.215346
   1    0.426410
d  2    0.916470
   3   -1.792631
dtype: float64
'''

#分類化索引,方便透過分類,讓我們取得資料更方便
#標籤索引
data['b']
'''
Out[2]: 
1    0.841650
2   -0.571143
dtype: float64
'''

#標籤slice,標籤名的slice,是有包含後面的標籤
data['b':'c']
'''
Out[3]: 
b  1    0.841650
   2   -0.571143
c  3    0.215346
   1    0.426410
dtype: float64
'''

#loc[list]
#[list]也可
data.loc[['b', 'd']]
'''
Out[4]: 
b  1    0.841650
   2   -0.571143
d  2    0.916470
   3   -1.792631
dtype: float64
'''

#loc[外部標籤,內部標籤]
data[:,2]
'''
Out[5]: 
a    0.914465
b   -0.571143
d    0.916470
dtype: float64
'''

#階層式索引最重要的目是是如下
#階層式索引可以運用在重新塑型資料和使用群組運算
#產生一個樞紐分析表
#使用unstack()方法塑型資料為DataFrame

data.unstack()
'''
Out[6]: 
          1         2         3
a  0.205758  0.914465 -1.129890
b  0.841650 -0.571143       NaN
c  0.426410       NaN  0.215346
d       NaN  0.916470 -1.792631
'''

#unstack()方法的相反的運算為stack()
data.unstack().stack()

'''
Out[2]: 
a  1    0.291485
   2    0.323402
   3    1.426483
b  1    1.447450
   2   -0.530195
c  1   -0.533092
   3   -1.438271
d  2    1.127549
   3   -0.523675
dtype: float64
'''

In [ ]:
#DataFrame使用多階層索引

import numpy as np
import pandas as pd

frame = pd.DataFrame(np.arange(12).reshape((4, 3)),
                    index=[['a','a','b','b'],[1, 2, 1, 2]],
                    columns=[['台北','台北','台中'],['Green','Red','Green']])
frame
'''
Out[7]: 
       台北        台中
    Green Red Green
a 1     0   1     2
  2     3   4     5
b 1     6   7     8
  2     9  10    11
'''

#階層式的索引可以命名,使用names,指定值使用list
frame.index.names = ['key1', 'key2']
frame.columns.names = ['縣市','顏色']
frame
'''
Out[8]: 
縣市           台北        台中
顏色        Green Red Green
key1 key2                
a    1        0   1     2
     2        3   4     5
b    1        6   7     8
     2        9  10    11
'''

#欄位索引
frame['台北']
'''
Out[9]: 
顏色         Green  Red
key1 key2            
a    1         0    1
     2         3    4
b    1         6    7
     2         9   10
'''

#可以使用pandas的MultiIndex可以獨自建立
pd.MultiIndex.from_arrays([['台北','台北','台中'],['Green','Red','Green']],
                       names=['縣市','顏色'])
'''
Out[10]: 
MultiIndex([('台北', 'Green'),
            ('台北',   'Red'),
            ('台中', 'Green')],
           names=['縣市', '顏色'])
'''

### 排列位置和排序
- 重新排列階層位置和依內容排序

In [ ]:
#swaplevel()
#sort_index()

import numpy as np
import pandas as pd

frame = pd.DataFrame(np.arange(12).reshape((4, 3)),
                    index=[['a','a','b','b'],[1, 2, 1, 2]],
                    columns=[['台北','台北','台中'],['Green','Red','Green']])
frame.index.names = ['key1', 'key2']
frame.columns.names = ['縣市','顏色']
frame
'''
Out[11]: 
縣市           台北        台中
顏色        Green Red Green
key1 key2                
a    1        0   1     2
     2        3   4     5
b    1        6   7     8
     2        9  10    11
'''

#使用swaplevel()方法對調索引
frame.swaplevel('key1','key2')
'''
Out[12]: 
縣市           台北        台中
顏色        Green Red Green
key2 key1                
1    a        0   1     2
2    a        3   4     5
1    b        6   7     8
2    b        9  10    11
'''

#使用sort_index(),依索引內容排序,其它資料內容會跟著移動

frame.sort_index(level=0)
'''
Out[13]: 
縣市           台北        台中
顏色        Green Red Green
key1 key2                
a    1        0   1     2
     2        3   4     5
b    1        6   7     8
     2        9  10    11
'''

frame.sort_index(level=1)
'''
Out[14]: 
縣市           台北        台中
顏色        Green Red Green
key1 key2                
a    1        0   1     2
b    1        6   7     8
a    2        3   4     5
b    2        9  10    11


'''

#同時交換和排序內容
frame.swaplevel(0,1).sort_index(level=0)
'''
Out[15]: 
縣市           台北        台中
顏色        Green Red Green
key2 key1                
1    a        0   1     2
     b        6   7     8
2    a        3   4     5
     b        9  10    11
'''

### 透過level的加總統計

In [ ]:
frame
'''
Out[16]: 
縣市           台北        台中
顏色        Green Red Green
key1 key2                
a    1        0   1     2
     2        3   4     5
b    1        6   7     8
     2        9  10    11
'''

#依照列內索引階層內的相同值做加總
#使用sum()方法,有引數名稱level,axis
frame.sum(level='key1')
'''
Out[17]: 
縣市      台北        台中
顏色   Green Red Green
key1                
a        3   5     7
b       15  17    19
'''

#依照欄內索引階層內的相同值做加總
frame.sum(level='顏色', axis=1)
'''
Out[18]: 
顏色         Green  Red
key1 key2            
a    1         2    1
     2         8    4
b    1        14    7
     2        20   10
'''



### 將DataFrame的欄位變為索引

In [ ]:
frame = pd.DataFrame({'a':range(7), 'b':range(7, 0, -1),
                     'c':['one', 'one', 'one', 'two', 'two', 'two', 'two'],
                     'd':[0, 1, 2, 0, 1, 2, 3]})
frame
'''
Out[4]: 
   a  b    c  d
0  0  7  one  0
1  1  6  one  1
2  2  5  one  2
3  3  4  two  0
4  4  3  two  1
5  5  2  two  2
6  6  1  two  3
'''

#將欄位的值成為列索引
frame2 = frame.set_index(['c' ,'d'])
frame2
'''
Out[5]: 
       a  b
c   d      
one 0  0  7
    1  1  6
    2  2  5
two 0  3  4
    1  4  3
    2  5  2
    3  6  1
'''

#使用引數名稱drop=False,保留欄位
frame.set_index(['c','d'], drop=False)
'''
Out[6]: 
       a  b    c  d
c   d              
one 0  0  7  one  0
    1  1  6  one  1
    2  2  5  one  2
two 0  3  4  two  0
    1  4  3  two  1
    2  5  2  two  2
    3  6  1  two  3
'''

#使用reset_index(),將索引變為欄位
frame2.reset_index()
'''
Out[7]: 
     c  d  a  b
0  one  0  0  7
1  one  1  1  6
2  one  2  2  5
3  two  0  3  4
4  two  1  4  3
5  two  2  5  2
6  two  3  6  1
'''

[homework1](https://github.com/roberthsu2003/PythonForDataAnalysis/blob/master/數據整理_連接_合併_重塑/homework1.ipynb)

## 連接和合併資料集
- 使用下列幾種方法  
    1 pandas.merge()依照欄位內的值連結其它的DataFrame相同欄位值的其它資料(類似關連式資料庫的join)  
    2 pandas.concat() 依axis連接資料集  
    3 combine_first()實體方法可以將有重疊部份的資料,覆蓋起來。 

### 類似於關聯資料庫的Join
- 2個資料集透過key(value)連結在一起

In [ ]:
df1 = pd.DataFrame({'key':['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                   'data1':range(7)})
df2 = pd.DataFrame({'key':['a', 'b', 'd'],
                   'data2':range(3)})
df1
'''
Out[8]: 
  key  data1
0   b      0
1   b      1
2   a      2
3   c      3
4   a      4
5   a      5
6   b      6
'''

df2
'''
Out[9]: 
  key  data2
0   a      0
1   b      1
2   d      2
'''

#多對1的連結
#沒有指定使用那一個欄位連結,則會以相同的欄位名當連結的依據
pd.merge(df1, df2)
'''
Out[10]: 
  key  data1  data2
0   b      0      1
1   b      1      1
2   b      6      1
3   a      2      0
4   a      4      0
5   a      5      0
'''

#有指定欄位的連結
pd.merge(df1, df2, on='key')

'''
Out[11]: 
  key  data1  data2
0   b      0      1
1   b      1      1
2   b      6      1
3   a      2      0
4   a      4      0
5   a      5      0
'''

#如果2個資料集的資料不一樣,可以指定2個資料集的欄位
df3 = pd.DataFrame({'lkey':['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                   'data1':range(7)})

df4 = pd.DataFrame({'rkey':['a', 'b', 'd'],
                   'data2':range(3)})

#inner join
pd.merge(df3, df4, left_on='lkey', right_on='rkey')
'''
Out[12]: 
  lkey  data1 rkey  data2
0    b      0    b      1
1    b      1    b      1
2    b      6    b      1
3    a      2    a      0
4    a      4    a      0
5    a      5    a      0
'''

#outer join
pd.merge(df3, df4, left_on='lkey', right_on='rkey', how='outer')
'''
Out[13]: 
  lkey  data1 rkey  data2
0    b    0.0    b    1.0
1    b    1.0    b    1.0
2    b    6.0    b    1.0
3    a    2.0    a    0.0
4    a    4.0    a    0.0
5    a    5.0    a    0.0
6    c    3.0  NaN    NaN
7  NaN    NaN    d    2.0
'''

#多對多的join
df1 = pd.DataFrame({'key':['b', 'b', 'a', 'c', 'a',  'b'],
                   'data1':range(6)})
df2 = pd.DataFrame({'key':['a', 'b', 'a', 'b', 'd'],
                   'data2':range(5)})
df1
'''
Out[14]: 
  key  data1
0   b      0
1   b      1
2   a      2
3   c      3
4   a      4
5   b      5
'''

df2
'''
Out[15]: 
  key  data2
0   a      0
1   b      1
2   a      2
3   b      3
4   d      4
'''

pd.merge(df1, df2, on='key', how='left')
'''
Out[16]: 
   key  data1  data2
0    b      0    1.0
1    b      0    3.0
2    b      1    1.0
3    b      1    3.0
4    a      2    0.0
5    a      2    2.0
6    c      3    NaN
7    a      4    0.0
8    a      4    2.0
9    b      5    1.0
10   b      5    3.0
'''

#多對多的inner join,沒有的欄位將會被排除
pd.merge(df1, df2, how='inner')
'''
Out[17]: 
  key  data1  data2
0   b      0      1
1   b      0      3
2   b      1      1
3   b      1      3
4   b      5      1
5   b      5      3
6   a      2      0
7   a      2      2
8   a      4      0
9   a      4      2
'''

### 使用索引merge
- left_index=True
- right_index=True



In [ ]:
import numpy as np
import pandas as pd

left1 = pd.DataFrame({'key':['a', 'b', 'a', 'a', 'b', 'c'],
                     'value':range(6)})

right1 = pd.DataFrame({'group_val':[3.5, 7]}, index=['a', 'b'])
left1
'''
Out[1]: 
  key  value
0   a      0
1   b      1
2   a      2
3   a      3
4   b      4
5   c      5

'''

right1
'''
Out[4]: 
   group_val
a        3.5
b        7.0
'''

#inner join可看成2個資料集的交集
pd.merge(left1, right1, left_on='key', right_index=True)
'''
Out[5]: 
  key  value  group_val
0   a      0        3.5
2   a      2        3.5
3   a      3        3.5
1   b      1        7.0
4   b      4        7.0
'''

#outer join可看成2個資料集的聯集 
pd.merge(left1, right1, left_on='key', right_index=True, how='outer')
'''
Out[6]: 
  key  value  group_val
0   a      0        3.5
2   a      2        3.5
3   a      3        3.5
1   b      1        7.0
4   b      4        7.0
5   c      5        NaN
'''

#merge()使用多階層索引的join
lefth = pd.DataFrame({'key1':['台北','台北','台北','台中','台中'],
                     'key2':[2000, 2001, 2002, 2001, 2002],
                    'data':np.arange(5.)})
lefth
'''
Out[7]: 
  key1  key2  data
0   台北  2000   0.0
1   台北  2001   1.0
2   台北  2002   2.0
3   台中  2001   3.0
4   台中  2002   4.0
'''

righth = pd.DataFrame(np.arange(12).reshape((6, 2)),
                     index=[['台中','台中','台北','台北','台北','台北'],[2001,2000, 2000, 2000,2001, 2002]],
                            columns=['event1','event2'])
righth
'''
Out[8]: 
         event1  event2
台中 2001       0       1
   2000       2       3
台北 2000       4       5
   2000       6       7
   2001       8       9
   2002      10      11
'''

#多欄位和多階層索引的連結,left_on做用list,right_index=True
pd.merge(lefth, righth, left_on=['key1','key2'], right_index=True)
'''
Out[9]: 
  key1  key2  data  event1  event2
0   台北  2000   0.0       4       5
0   台北  2000   0.0       6       7
1   台北  2001   1.0       8       9
2   台北  2002   2.0      10      11
3   台中  2001   3.0       0       1
'''

#左,右都使用索引值連結
left2 = pd.DataFrame([[1., 2.],[3., 4.],[5., 6.]],
                    index=['a', 'c', 'e'],
                    columns=['台北','台中'])

right2 = pd.DataFrame([[7., 8.],[9., 10.],[11., 12.],[13., 14]],
                    index=['b', 'c', 'd', 'e'],
                    columns=['花蓮','屏東'])
left2
'''
Out[10]: 
    台北   台中
a  1.0  2.0
c  3.0  4.0
e  5.0  6.0
'''

right2
'''
Out[11]: 
     花蓮    屏東
b   7.0   8.0
c   9.0  10.0
d  11.0  12.0
e  13.0  14.0
'''
pd.merge(left2, right2, how='outer', left_index=True, right_index=True)
'''
    台北   台中    花蓮    屏東
a  1.0  2.0   NaN   NaN
b  NaN  NaN   7.0   8.0
c  3.0  4.0   9.0  10.0
d  NaN  NaN  11.0  12.0
e  5.0  6.0  13.0  14.0

'''

#使用實體方法join(),只能使用自已本身的index值做連結
left2.join(right2, how='outer')
'''
Out[13]: 
    台北   台中    花蓮    屏東
a  1.0  2.0   NaN   NaN
b  NaN  NaN   7.0   8.0
c  3.0  4.0   9.0  10.0
d  NaN  NaN  11.0  12.0
e  5.0  6.0  13.0  14.0
'''

#join(),另一個資料集可以使用column name
left1.join(right1, on='key')
'''
Out[14]: 
  key  value  group_val
0   a      0        3.5
1   b      1        7.0
2   a      2        3.5
3   a      3        3.5
4   b      4        7.0
5   c      5        NaN
'''



### 以軸為連結的依據
- numpy np.concatenate()
- pandas pd.concat()



In [ ]:
arr = np.arange(12).reshape((3, 4))
arr
'''
Out[15]: 
array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])
'''
#axis=1,想像擴增欄
np.concatenate([arr, arr], axis=1)
'''
Out[16]: 
array([[ 0,  1,  2,  3,  0,  1,  2,  3],
       [ 4,  5,  6,  7,  4,  5,  6,  7],
       [ 8,  9, 10, 11,  8,  9, 10, 11]])
'''


#### DataFrame的concatenate要注意的地方
- 如果連結的軸的索引是否相同
- 是否連結的資料最後產生的結果必需被識別


In [20]:
#Series的concate, 每個軸有不同的index
import numpy as np
import pandas as pd

s1 = pd.Series([0, 1], index=['a', 'b'])
s2 = pd.Series([2, 3, 4], index=['c', 'd', 'e'])
s3 = pd.Series([5, 6], index=['f', 'g'])
pd.concat([s1, s2, s3])

'''
Out[17]: 
a    0
b    1
c    2
d    3
e    4
f    5
g    6
dtype: int64
'''

#預設的concat是axis=0(擴充列),axis=1(擴充欄)是欄的concate
pd.concat([s1, s2, s3], axis=1, sort=False)
'''
Out[18]: 
     0    1    2
a  0.0  NaN  NaN
b  1.0  NaN  NaN
c  NaN  2.0  NaN
d  NaN  3.0  NaN
e  NaN  4.0  NaN
f  NaN  NaN  5.0
g  NaN  NaN  6.0
'''

#類似join()
s4 = pd.concat([s1, s3])
s4
'''
Out[19]: 
a    0
b    1
f    5
g    6
dtype: int64
'''

s1
'''
Out[20]: 
a    0
b    1
dtype: int64
'''
pd.concat([s1, s4],axis=1,sort=False)
'''
Out[21]: 
     0  1
a  0.0  0
b  1.0  1
f  NaN  5
g  NaN  6
'''

#使用引數名稱join='inner'
pd.concat([s1, s4],axis=1, join='inner')
'''
Out[22]: 
   0  1
a  0  0
b  1  1
'''



#concat後,結果必需初識別
result = pd.concat([s1, s1, s3], keys=['one', 'two', 'three'])
result
'''
Out[3]: 
one    a    0
       b    1
two    a    0
       b    1
three  f    5
       g    6
dtype: int64
'''

result.unstack()
'''
Out[4]: 
         a    b    f    g
one    0.0  1.0  NaN  NaN
two    0.0  1.0  NaN  NaN
three  NaN  NaN  5.0  6.0
'''

#concat後,結果必需初識別
df1 = pd.DataFrame(np.arange(6).reshape(3, 2),
                   index=['a', 'b', 'c'],
                   columns=['one','two'])
df2 = pd.DataFrame(np.arange(4).reshape(2, 2),
                   index=['a', 'c'],
                   columns=['three','four'])

df1
'''
Out[5]: 
   one  two
a    0    1
b    2    3
c    4    5
'''

df2
'''
Out[6]: 
   three  four
a      0     1
c      2     3
'''

pd.concat([df1, df2], axis=1, keys=['level1', 'level2'], sort=False)
'''
Out[7]: 
  level1     level2     
     one two  three four
a      0   1    0.0  1.0
b      2   3    NaN  NaN
c      4   5    2.0  3.0
'''

#使用names,指定欄索引名稱
pd.concat([df1, df2], axis=1, 
          keys=['level1', 'level2'], 
          names=['upper', 'lower'],
          sort=False)

#使用引數名稱ignore_index = True, 排除如果有相同的列索引的使用
df1 = pd.DataFrame(np.random.randn(3, 4), columns=['a', 'b', 'c', 'd'])
df2 = pd.DataFrame(np.random.randn(2, 3), columns=['b', 'd', 'a'])
df1
'''
Out[9]: 
          a         b         c         d
0 -0.292134  0.219706 -1.267903  0.168186
1  1.492984 -1.019931  0.594643 -0.342395
2  1.427346  0.141581 -2.120926  0.854317
'''

df2
'''
Out[10]: 
          b         d         a
0  2.024582  0.809604 -0.466355
1 -0.644573  0.998421  0.549668
'''

pd.concat([df1, df2],sort=False)
'''
Out[11]: 
          a         b         c         d
0 -0.292134  0.219706 -1.267903  0.168186
1  1.492984 -1.019931  0.594643 -0.342395
2  1.427346  0.141581 -2.120926  0.854317
0 -0.466355  2.024582       NaN  0.809604
1  0.549668 -0.644573       NaN  0.998421
'''

pd.concat([df1, df2],sort=False,ignore_index=True)
'''
Out[12]: 
          a         b         c         d
0 -0.292134  0.219706 -1.267903  0.168186
1  1.492984 -1.019931  0.594643 -0.342395
2  1.427346  0.141581 -2.120926  0.854317
3 -0.466355  2.024582       NaN  0.809604
4  0.549668 -0.644573       NaN  0.998421
'''

,a,b,c,d
0,-0.874814,-0.034552,-0.114859,-1.062524
1,-0.671357,-1.998374,-0.520020,-1.090595
2,0.304228,-1.109316,-0.959026,0.580322
3,0.903273,-0.404049,NaN,-0.305312
4,-1.894440,0.786586,NaN,1.331132


#### Homework2
[homework2](https://github.com/roberthsu2003/PythonForDataAnalysis/blob/master/數據整理_連接_合併_重塑/homework2.ipynb)

#### Homework3  
[homework3](https://github.com/roberthsu2003/PythonForDataAnalysis/blob/master/數據整理_連接_合併_重塑/homework3.ipynb)